# Market Basket Analysis

Market basket analysis scrutinizes the products customers tend to buy together, and uses the information to decide which products should be cross-sold or promoted together. The term arises from the shopping carts supermarket shoppers fill up during a shopping trip.

Association Rule Mining is used when we want to find an association between different objects in a set, find frequent patterns in a transaction database, relational databases or any other information repository.

The most common approach to find these patterns is Market Basket Analysis, which is a key technique used by large retailers like Amazon, Flipkart, etc to analyze customer buying habits by finding associations between the different items that customers place in their “shopping baskets”. The discovery of these associations can help retailers develop marketing strategies by gaining insight into which items are frequently purchased together by customers. The strategies may include:

- Changing the store layout according to trends
- Customers behavior analysis
- Catalog Design
- Cross marketing on online stores
- Customized emails with add-on sales, etc.

In [1]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

#import utility functions from script
from utils import *

#Data file directory
root = 'C:/Users/ALEXANDER/Documents/Machine Learning/Instacart/Data/'

In [2]:
#Read datasets
orders = read(root,path='orders')
order_products_prior = read(root,path='order_products__prior')
order_products_train = read(root,path='order_products__train')
products = read(root,path="products")

In [3]:
combined_order = pd.concat([order_products_prior,order_products_train],0)
combined_order.head()

order_id  product_id  add_to_cart_order  reordered
0         2       33120                  1          1
1         2       28985                  2          1
2         2        9327                  3          0
3         2       45918                  4          1
4         2       30035                  5          0

In [4]:
print(f"There are {combined_order['product_id'].nunique()} unique products.")


There are 49685 unique products.


- Since the gaol of Market Basket Analysis is to find relationship between products which are frequently bought together. Our analysis will be based on the top 100 frequent products.

In [5]:
product_counts = combined_order.groupby('product_id')['order_id'].count().reset_index().rename(columns = {'order_id':'frequency'})
product_counts = product_counts.sort_values('frequency', ascending=False)[:100].reset_index(drop = True)
product_counts = product_counts.merge(products, on = 'product_id', how = 'left')
product_counts.head(10)

product_id  frequency            product_name  aisle_id  department_id
0       24852     491291                  Banana        24              4
1       13176     394930  Bag of Organic Bananas        24              4
2       21137     275577    Organic Strawberries        24              4
3       21903     251705    Organic Baby Spinach       123              4
4       47209     220877    Organic Hass Avocado        24              4
5       47766     184224         Organic Avocado        24              4
6       47626     160792             Large Lemon        24              4
7       16797     149445            Strawberries        24              4
8       26209     146660                   Limes        24              4
9       27845     142813      Organic Whole Milk        84             16

In [6]:
freq_products = list(product_counts.product_id)

#Get order for only top 100 frequently occuring products
combined_order = combined_order[combined_order['product_id'].isin(freq_products)]
combined_order.shape

(7795471, 4)

In [7]:
#Get metadata information of these top 100 products
combined_order = combined_order.merge(products, on = 'product_id', how='left')
combined_order.head()

order_id  product_id  add_to_cart_order  reordered            product_name  \
0         2       28985                  2          1   Michigan Organic Kale   
1         2       17794                  6          1                 Carrots   
2         3       24838                  2          1  Unsweetened Almondmilk   
3         3       21903                  4          1    Organic Baby Spinach   
4         3       46667                  6          1     Organic Ginger Root   

   aisle_id  department_id  
0        83              4  
1        83              4  
2        91             16  
3       123              4  
4        83              4

Next, I will create a basket: This represents a binary matrix, this tells us which products were bought together for every order in the dataset.
for instance, given a binary matrix representing the products present in an order:

====> 1    0    0    0    0   1

This means that only product 1 and 6 were bought together for this specific order.

In [8]:
basket = combined_order.groupby(['order_id', 'product_name'])['reordered'].count().unstack().reset_index().fillna(0).set_index('order_id')
basket.head()

product_name  100% Raw Coconut Water  100% Whole Wheat Bread  \
order_id                                                       
1                                0.0                     0.0   
2                                0.0                     0.0   
3                                0.0                     0.0   
5                                0.0                     0.0   
9                                0.0                     0.0   

product_name  2% Reduced Fat Milk  Apple Honeycrisp Organic  Asparagus  \
order_id                                                                 
1                             0.0                       0.0        0.0   
2                             0.0                       0.0        0.0   
3                             0.0                       0.0        0.0   
5                             1.0                       0.0        0.0   
9                             0.0                       0.0        0.0   

product_name  Bag of Organic Bananas  Banana  Bartlett Pears  Blueberries  \
order_id                                                                    
1                                1.0     0.0             0.0          0.0   
2                                0.0     0.0             0.0          0.0   
3                                0.0     0.0             0.0          0.0   
5                                1.0     0.0             0.0          0.0   
9                                0.0     0.0             0.0          0.0   

product_name  Boneless Skinless Chicken Breasts  Broccoli Crown  \
order_id                                                          
1                                           0.0             0.0   
2                                           0.0             0.0   
3                                           0.0             0.0   
5                                           0.0             0.0   
9                                           0.0             0.0   

product_name  Bunched Cilantro  Carrots  Clementines, Bag  Cucumber Kirby  \
order_id                                                                    
1                          0.0      0.0               0.0             1.0   
2                          0.0      1.0               0.0             0.0   
3                          0.0      0.0               0.0             0.0   
5                          0.0      0.0               0.0             0.0   
9                          0.0      0.0               0.0             0.0   

product_name  Extra Virgin Olive Oil  Fresh Cauliflower  Garlic  \
order_id                                                          
1                                0.0                0.0     0.0   
2                                0.0                0.0     0.0   
3                                0.0                0.0     0.0   
5                                0.0                0.0     0.0   
9                                1.0                0.0     0.0   

product_name  Granny Smith Apples  Grape White/Green Seedless  \
order_id                                                        
1                             0.0                         0.0   
2                             0.0                         0.0   
3                             0.0                         0.0   
5                             0.0                         0.0   
9                             0.0                         0.0   

product_name  Grated Parmesan  Green Bell Pepper  Half & Half  Hass Avocados  \
order_id                                                                       
1                         0.0                0.0          0.0            0.0   
2                         0.0                0.0          0.0            0.0   
3                         0.0                0.0          0.0            0.0   
5                         0.0                0.0          0.0            0.0   
9                         0.0                0.0          0.0            0.0

- Free memory usage by deleting some variables from memory.

In [9]:
#Free memory usage
del product_counts, products, combined_order, order_products_prior, order_products_train

In [10]:
#Round off units in basket matrix
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1 
    
basket = basket.applymap(encode_units)
basket.head()

product_name  100% Raw Coconut Water  100% Whole Wheat Bread  \
order_id                                                       
1                                  0                       0   
2                                  0                       0   
3                                  0                       0   
5                                  0                       0   
9                                  0                       0   

product_name  2% Reduced Fat Milk  Apple Honeycrisp Organic  Asparagus  \
order_id                                                                 
1                               0                         0          0   
2                               0                         0          0   
3                               0                         0          0   
5                               1                         0          0   
9                               0                         0          0   

product_name  Bag of Organic Bananas  Banana  Bartlett Pears  Blueberries  \
order_id                                                                    
1                                  1       0               0            0   
2                                  0       0               0            0   
3                                  0       0               0            0   
5                                  1       0               0            0   
9                                  0       0               0            0   

product_name  Boneless Skinless Chicken Breasts  Broccoli Crown  \
order_id                                                          
1                                             0               0   
2                                             0               0   
3                                             0               0   
5                                             0               0   
9                                             0               0   

product_name  Bunched Cilantro  Carrots  Clementines, Bag  Cucumber Kirby  \
order_id                                                                    
1                            0        0                 0               1   
2                            0        1                 0               0   
3                            0        0                 0               0   
5                            0        0                 0               0   
9                            0        0                 0               0   

product_name  Extra Virgin Olive Oil  Fresh Cauliflower  Garlic  \
order_id                                                          
1                                  0                  0       0   
2                                  0                  0       0   
3                                  0                  0       0   
5                                  0                  0       0   
9                                  1                  0       0   

product_name  Granny Smith Apples  Grape White/Green Seedless  \
order_id                                                        
1                               0                           0   
2                               0                           0   
3                               0                           0   
5                               0                           0   
9                               0                           0   

product_name  Grated Parmesan  Green Bell Pepper  Half & Half  Hass Avocados  \
order_id                                                                       
1                           0                  0            0              0   
2                           0                  0            0              0   
3                           0                  0            0              0   
5                           0                  0            0              0   
9                           0                  0            0              0

### TERMINOLOGIES

- **Support** : Its the default popularity of an item. In mathematical terms, the support of item A is the ratio of transactions involving A to the total number of transactions.


- **Confidence** : Likelihood that customer will buy both products A and B. It is the ratio of the number of transactions involving both A and B and the number of transactions involving B.
     - Confidence(A => B) = Support(A, B)/Support(A)


- **Lift** : Increase in the sale of A when you sell B.
    
    - Lift(A => B) = Confidence(A, B)/Support(B)
        
    - Lift (A => B) = 1 means that there is no correlation within the itemset.
    - Lift (A => B) > 1 means that there is a positive correlation within the itemset, i.e., products in the itemset, A, and B, are more likely to be bought together.
    - Lift (A => B) < 1 means that there is a negative correlation within the itemset, i.e., products in itemset, A, and B, are unlikely to be bought together.

**Apriori Algorithm:** refers to the algorithm which is used to calculate the association rules between objects. It means how two or more objects are related to one another. In other words, we can say that the apriori algorithm is an association rule leaning that analyzes that people who bought product A also bought product B.

Apriori algorithm assumes that any subset of a frequent itemset must be frequent. Its the algorithm behind Market Basket Analysis. Say, a transaction containing {Grapes, Apple, Mango} also contains {Grapes, Mango}. So, according to the principle of Apriori, if {Grapes, Apple, Mango} is frequent, then {Grapes, Mango} must also be frequent.

In [11]:
frequent_items = apriori(basket, min_support=0.01, use_colnames=True, low_memory=True)
frequent_items.head()

C:\Users\ALEXANDER\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


support                    itemsets
0  0.016062    (100% Raw Coconut Water)
1  0.025814    (100% Whole Wheat Bread)
2  0.015800       (2% Reduced Fat Milk)
3  0.035694  (Apple Honeycrisp Organic)
4  0.029101                 (Asparagus)

In [12]:
rules = association_rules(frequent_items, metric="lift", min_threshold=1)
final_results = rules.sort_values('lift', ascending=False)

In [13]:
final_results

antecedents               consequents  antecedent support  \
35                   (Limes)             (Large Lemon)            0.059984   
34             (Large Lemon)                   (Limes)            0.065764   
52    (Organic Strawberries)     (Organic Raspberries)            0.112711   
53     (Organic Raspberries)    (Organic Strawberries)            0.058325   
37         (Organic Avocado)             (Large Lemon)            0.075348   
36             (Large Lemon)         (Organic Avocado)            0.065764   
47     (Organic Blueberries)    (Organic Strawberries)            0.042956   
46    (Organic Strawberries)     (Organic Blueberries)            0.112711   
49     (Organic Raspberries)    (Organic Hass Avocado)            0.058325   
48    (Organic Hass Avocado)     (Organic Raspberries)            0.090339   
24                  (Banana)      (Organic Fuji Apple)            0.200938   
25      (Organic Fuji Apple)                  (Banana)            0.037992   
4   (Bag of Organic Bananas)     (Organic Raspberries)            0.161527   
5      (Organic Raspberries)  (Bag of Organic Bananas)            0.058325   
3   (Bag of Organic Bananas)    (Organic Hass Avocado)            0.161527   
2     (Organic Hass Avocado)  (Bag of Organic Bananas)            0.090339   
14        (Honeycrisp Apple)                  (Banana)            0.034078   
15                  (Banana)        (Honeycrisp Apple)            0.200938   
38         (Organic Avocado)    (Organic Baby Spinach)            0.075348   
39    (Organic Baby Spinach)         (Organic Avocado)            0.102948   
51    (Organic Strawberries)    (Organic Hass Avocado)            0.112711   
50    (Organic Hass Avocado)    (Organic Strawberries)            0.090339   
12          (Cucumber Kirby)                  (Banana)            0.040789   
13                  (Banana)          (Cucumber Kirby)            0.200938   
42    (Organic Hass Avocado)    (Organic Baby Spinach)            0.090339   
43    (Organic Baby Spinach)    (Organic Hass Avocado)            0.102948   
55      (Organic Whole Milk)    (Organic Strawberries)            0.058411   
54    (Organic Strawberries)      (Organic Whole Milk)            0.112711   
20         (Organic Avocado)                  (Banana)            0.075348   
21                  (Banana)         (Organic Avocado)            0.200938   
31     (Seedless Red Grapes)                  (Banana)            0.035480   
30                  (Banana)     (Seedless Red Grapes)            0.200938   
7   (Bag of Organic Bananas)    (Organic Strawberries)            0.161527   
6     (Organic Strawberries)  (Bag of Organic Bananas)            0.112711   
32            (Strawberries)                  (Banana)            0.061123   
33                  (Banana)            (Strawberries)            0.200938   
45    (Organic Baby Spinach)    (Organic Strawberries)            0.102948   
44    (Organic Strawberries)    (Organic Baby Spinach)            0.112711   
10  (Bag of Organic Bananas)    (Organic Yellow Onion)            0.161527   
11    (Organic Yellow Onion)  (Bag of Organic Bananas)            0.048146   
16             (Large Lemon)                  (Banana)            0.065764   
17                  (Banana)             (Large Lemon)            0.200938   
0   (Bag of Organic Bananas)    (Organic Baby Spinach)            0.161527   
1     (Organic Baby Spinach)  (Bag of Organic Bananas)            0.102948   
40         (Organic Avocado)    (Organic Strawberries)            0.075348   
41    (Organic Strawberries)         (Organic Avocado)            0.112711   
9   (Bag of Organic Bananas)      (Organic Whole Milk)            0.161527   
8       (Organic Whole Milk)  (Bag of Organic Bananas)            0.058411   
29      (Organic Whole Milk)                  (Banana)            0.058411   
28                  (Banana)      (Organic Whole Milk)            0.200938   
19                  (Banana)           

- Antecedent: refers to the items on the LEFT ie., the item which the customer buy
- Consequent: refers to the items on the RIGHT ie., the item which the customer follows to buy.

### Inference:

- From the results it’s very clear that, higher the LIFT value, stronger the correlation between the antecedents and the consequents.
    - A lift greater than 1 suggests that the presence of the antecedent increases the chances that the consequent will occur in a given transaction.
    - Lift below 1 indicates that purchasing the antecedent reduces the chances of purchasing the consequent in the same transaction. Note: This could indicate that the items are seen by customers as alternatives to each other.
    - When the lift is 1, then purchasing the antecedent makes no difference on the chances of purchasing the consequent. 
    
- Using Lift as our main performance measure, I could see that:

    - Whoever buys (`Limes`) ==>(also buys) (`Large lemon`) with the confidence of 19.77%, which means almost 19.77% of the transactions has limes and large lemon.. The LIFT is also around 3.006 ie., there is more chances to take lemon whenever they take lime. n these items, i.e., both products are more likely to be bought together.
    - Most organic fruits are likely to be bought together (as their lift score is greater than 1) and should be placed side by side for visibility in stores.